In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

#Parameters
ticker_symbol = "AAPL"
start = "2023-01-01"
end = "2024-01-01"

#Stock Data Retrieval
ticker = yf.Ticker(ticker_symbol)
historical_data = ticker.history(start=start, end=end)
closing_prices = historical_data['Close']
option_expirations = ticker.options

#Option Expiration
chosen_expiry = option_expirations[-1]  # Choose the last available expiration date
options_chain = ticker.option_chain(chosen_expiry)

#Strike And Option Prices
callables = options_chain.calls
putables = options_chain.puts
calls_data = callables[['strike', 'lastPrice']]
puts_data = putables[['strike', 'lastPrice']]

#Log Returns
log_returns = np.log(closing_prices / closing_prices.shift(1)).dropna()
daily_volatility = log_returns.std()
annualized_volatility = daily_volatility * np.sqrt(252)

S0 = closing_prices.iloc[-1] # Last closing price
K = calls_data['strike'].values # Strike prices of calls
T = (pd.to_datetime(chosen_expiry) - pd.to_datetime(end)).days / 365  # Time to expiration in years
r = 0.05  # Risk-free interest rate
sigma = annualized_volatility
N = 2 # Time steps

dt = T / N  # Time step size
u = np.exp(sigma * np.sqrt(dt))  # Up factor
d = 1 / u  # Down factor
p = (np.exp(r * dt) - d) / (u - d)

asset_tree = np.zeros((N + 1, N + 1))

for i in range(N + 1):
    for j in range(i + 1):
        asset_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

#Terminal Payoff At Maturity (Call Option)
option_payoff = np.zeros(N+ 1)
for j in range(N + 1):
    ST = asset_tree[j, N] # Stock price at maturity
    option_payoff[j] = max(0, ST - K[-1])  # Payoff for the last strike price

#Build The Full Return Matrix (Backward Induction)
option_tree = np.zeros_like(asset_tree)
for j in range(N + 1):
    option_tree[j, N] = option_payoff[j]  # Set terminal node values (payoffs)

for i in range(N - 1, -1, -1):
    for j in range(i + 1):
        option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])


